# Message Spam Filter with Naive Bayes Algorithm

In this project, we're going to build a SMS spam filter using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The UCI Machine Learning Repository. The data collection process is described in more details on this page, where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

In [24]:
#Import the python libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

## Exploring the Dataset
We'll start off by loading the 'SMSSpamCollection.csv' file into pandas dataframe. The csv file is tab seperated.

In [26]:
df=pd.read_csv('SMSSpamCollection',sep='\t',\
               header=None,names=['Label', 'SMS'])

print(df.shape)
df.head(10)

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [28]:
# Check proportion of spam and non-spam messages
df['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

### we see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

## Training and Test Set
We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [29]:
# Randomize the dataset before splitting into train and test datasets
dfrnd=df.sample(frac=1,random_state=1)

In [33]:
# Assign 80% of all data to train  dataset and remaining 
# 20% to test dataset
train_size=round(len(dfrnd) * 0.8)
traindf=dfrnd.iloc[: train_size]
testdf=dfrnd.iloc[train_size: ]

print(traindf.shape)
print(testdf.shape)

(4458, 2)
(1114, 2)


In [35]:
# Look at the spam and non-spam proportions in both dataframes
# if data are well randomized
print(traindf['Label'].value_counts(normalize=True)*100)
print(testdf['Label'].value_counts(normalize=True)*100)

ham     86.54105
spam    13.45895
Name: Label, dtype: float64
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


#### The results look good! We'll now move on to cleaning the dataset.

## Data Cleaning
To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

'SMS' column containing the message's body will be converted into a list of words used in the message. 

In [37]:
# Remove any character that is not from the set a-z, A-Z or 0-9.
# Transform every letter in every word to lower case.
# Split words at space character.
train_up=traindf.copy()
train_up['SMS']=traindf['SMS'].str.replace('\W',' ').str.lower()
train_up['SMS']=train_up['SMS'].str.split()
train_up['SMS'].head()


1078                    [yep, by, the, pretty, sculpture]
4028    [yes, princess, are, you, going, to, make, me,...
958                       [welp, apparently, he, retired]
4642                                             [havent]
4674    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

### Creating the Vocabulary
Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [52]:
# List of all words used in the training set
vocabulary=train_up['SMS'].sum()
word_count=len(vocabulary)
print("Number of all words used in the training set: {}".format(word_count))
      

# Set of distinct words
temp=set(vocabulary)
vocabulary=list(temp)
word_count_dist=len(vocabulary)
print("Number of distinct words used in the training set: {}".format(word_count_dist))

Number of all words used in the training set: 72427
Number of distinct words used in the training set: 7783


### The Final Training Set
##### We're now going to use the vocabulary we just created to make the data transformation we want.

In [53]:
# For each distinct word in training set, count the number of occurences
# in each message
for word in vocabulary:
    train_up[word]=train_up['SMS'].apply(lambda x: x.count(word))
    

In [55]:
train_up.iloc[:3,:5]

,Label,SMS,disclose,play,3optical
1078,ham,"[yep, by, the, pretty, sculpture]",0,0,0
4028,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0
958,ham,"[welp, apparently, he, retired]",0,0,0


### Calculating Constants First
We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

P(Spam) and P(Ham)
NSpam, NHam, NVocabulary
We'll also use Laplace smoothing and set $\alpha = 1$.

In [56]:
# P(Spam) and P(Ham)
p_spam=train_up['Label'].value_counts(normalize=True).loc['spam']
p_ham=train_up['Label'].value_counts(normalize=True).loc['ham']

# Isolating spam and ham messages
n_spam=len(train_up[train_up['Label']=='spam']['SMS'].sum())
n_ham=len(train_up[train_up['Label']=='ham']['SMS'].sum())

# N_Vocabulary
n_voc=len(vocabulary)

#Laplace smoothing
alpha=1


### Calculating Parameters
Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [57]:
# Calculate parameters
p_w_spam=dict()
p_w_ham=dict()
spamdf=train_up[train_up['Label']=='spam']
hamdf=train_up[train_up['Label']=='ham']
for word in vocabulary:    
    p_w_spam[word]=(spamdf[word].sum()+alpha)/(n_spam+alpha*n_voc)
    p_w_ham[word]=(hamdf[word].sum()+alpha)/(n_ham+alpha*n_voc)

In [16]:
test=vocabulary[:15]
print('Word :                p_w_spam      p_w_ham')
print('*'*50)
for word in test:
    
    print('{:20s}   {:.7f}    {:.7f}'.format(word,p_w_spam[word],p_w_ham[word]))

Word :                p_w_spam      p_w_ham
**************************************************
disclose               0.0000435    0.0000461
play                   0.0005224    0.0003076
3optical               0.0000435    0.0000308
unsecured              0.0000871    0.0000154
terrific               0.0000435    0.0000308
lay                    0.0000435    0.0000308
447797706009           0.0001306    0.0000154
club                   0.0007835    0.0000154
fuckin                 0.0000435    0.0000615
630                    0.0000435    0.0000461
bring                  0.0000435    0.0002922
hv9d                   0.0000871    0.0000154
delay                  0.0000435    0.0000461
suite342               0.0006094    0.0000154
apps                   0.0000435    0.0000461


##### All calculations above look fine!..

### Classifying A New Message
Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

Takes in as input a new message (w1, w2, ..., wn).
Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [17]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
   
#     This is where we calculate:
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

#     p_spam_given_message = ?
#     p_ham_given_message = ?

    for word in message:
        if word in vocabulary:
            p_spam_given_message*=p_w_spam[word]
            p_ham_given_message*=p_w_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [18]:
# Test out our classifier
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [58]:
# Test out our classifier
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Measuring the Spam Filter's Accuracy
The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [20]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
   
#     This is where we calculate:
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

#     p_spam_given_message = ?
#     p_ham_given_message = ?

    for word in message:
        if word in vocabulary:
            p_spam_given_message*=p_w_spam[word]
            p_ham_given_message*=p_w_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return('ham') 
    elif p_ham_given_message < p_spam_given_message:
        return('spam')
    else:
        return('Equal proabilities, have a human classify this!')


###### Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set

In [59]:
testdf_up=testdf.copy()
pred=testdf['SMS'].apply(classify_test_set)
testdf_up['predicted']=pred
testdf_up.head()

,Label,SMS,predicted
2131,ham,Later i guess. I needa do mcat study too.,ham
3418,ham,But i haf enuff space got like 4 mb...,ham
3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
5393,ham,"All done, all handed in. Don't know if mega sh...",ham


##### Now it is time to calculate the accuracy of our spam filter.

In [63]:
correct=testdf_up['predicted']==testdf_up['Label']
no_correct=correct.sum()
total=testdf_up.shape[0]
accuracy=no_correct/total
print('Correct:', no_correct)
print('Incorrect:', total - no_correct)
print('Accuracy: {:.3f}'.format(accuracy))

Correct: 1100
Incorrect: 14
Accuracy: 0.987


** The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly. **

### Next step:

Let's analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly.

In [64]:
in_corr=testdf_up['predicted']!=testdf_up['Label']

testdf_up[in_corr]

,Label,SMS,predicted
3460,spam,Not heard from U4 a while. Call me now am here...,ham
1940,spam,More people are dogging in your area now. Call...,ham
3890,ham,Unlimited texts. Limited minutes.,spam
991,ham,26th OF JULY,spam
4862,ham,Nokia phone is lovly..,spam
2370,ham,A Boy loved a gal. He propsd bt she didnt mind...,"Equal proabilities, have a human classify this!"
326,ham,No calls..messages..missed calls,spam
5046,ham,We have sent JD for Customer Service cum Accou...,spam
3864,spam,Oh my god! I've found your number again! I'm s...,ham
4676,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham


** Our spam classifier failed at 14 messages. It is obvious from the above table that even human eye can not distinguish them correctly. **

### Conclusion
In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

Next steps include:

Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly